In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:

import os
os.environ['TF_KERAS'] = '1'
!echo $TF_KERAS

$TF_KERAS


In [3]:
from keras_radam import RAdam

import librosa
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import IPython.display as ipd
import librosa.display
import scipy
import glob
import numpy as np
import math
import warnings
import pickle
from sklearn.utils import shuffle

# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard module is not an IPython extension.


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14878617181860567226, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3174592921
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10738967871256365166
 physical_device_desc: "device: 0, name: GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [5]:
train_tfrecords_filenames = glob.glob('cnTrain/*')
#np.random.shuffle(train_tfrecords_filenames)
train_tfrecords_filenames = list(train_tfrecords_filenames)

val_tfrecords_filenames = glob.glob('cnValid/*')
print(train_tfrecords_filenames)

['cnTrain\\clnsp1.wav', 'cnTrain\\clnsp10.wav', 'cnTrain\\clnsp2.wav', 'cnTrain\\clnsp3.wav', 'cnTrain\\clnsp4.wav', 'cnTrain\\clnsp5.wav', 'cnTrain\\clnsp6.wav', 'cnTrain\\clnsp7.wav', 'cnTrain\\clnsp8.wav', 'cnTrain\\clnsp9.wav', 'cnTrain\\noisy10_SNRdb_0.0_clnsp10.wav', 'cnTrain\\noisy10_SNRdb_10.0_clnsp10.wav', 'cnTrain\\noisy10_SNRdb_20.0_clnsp10.wav', 'cnTrain\\noisy10_SNRdb_30.0_clnsp10.wav', 'cnTrain\\noisy10_SNRdb_40.0_clnsp10.wav', 'cnTrain\\noisy1_SNRdb_0.0_clnsp1.wav', 'cnTrain\\noisy1_SNRdb_10.0_clnsp1.wav', 'cnTrain\\noisy1_SNRdb_20.0_clnsp1.wav', 'cnTrain\\noisy1_SNRdb_30.0_clnsp1.wav', 'cnTrain\\noisy1_SNRdb_40.0_clnsp1.wav', 'cnTrain\\noisy2_SNRdb_0.0_clnsp2.wav', 'cnTrain\\noisy2_SNRdb_10.0_clnsp2.wav', 'cnTrain\\noisy2_SNRdb_20.0_clnsp2.wav', 'cnTrain\\noisy2_SNRdb_30.0_clnsp2.wav', 'cnTrain\\noisy2_SNRdb_40.0_clnsp2.wav', 'cnTrain\\noisy3_SNRdb_0.0_clnsp3.wav', 'cnTrain\\noisy3_SNRdb_10.0_clnsp3.wav', 'cnTrain\\noisy3_SNRdb_20.0_clnsp3.wav', 'cnTrain\\noisy3_SNRdb_3

In [6]:
windowLength = 256
overlap      = round(0.25 * windowLength) # overlap of 75%
ffTLength    = windowLength
inputFs      = 48e3
fs           = 16e3
numFeatures  = ffTLength//2 + 1
numSegments  = 8
print("windowLength:",windowLength)
print("overlap:",overlap)
print("ffTLength:",ffTLength)
print("inputFs:",inputFs)
print("fs:",fs)
print("numFeatures:",numFeatures)
print("numSegments:",numSegments)

windowLength: 256
overlap: 64
ffTLength: 256
inputFs: 48000.0
fs: 16000.0
numFeatures: 129
numSegments: 8


In [7]:
def tf_record_parser(record):
    keys_to_features = {
        "noise_stft_phase": tf.io.FixedLenFeature((), tf.string, default_value=""),
        'noise_stft_mag_features': tf.io.FixedLenFeature([], tf.string),
        "clean_stft_magnitude": tf.io.FixedLenFeature((), tf.string)
    }
    

    features = tf.io.parse_single_example(record, keys_to_features)
    print("features: ", features)

    noise_stft_mag_features = tf.io.decode_raw(features['noise_stft_mag_features'], tf.float32)
    clean_stft_magnitude = tf.io.decode_raw(features['clean_stft_magnitude'], tf.float32)
    noise_stft_phase = tf.io.decode_raw(features['noise_stft_phase'], tf.float32)

    # reshape input and annotation images
    noise_stft_mag_features = tf.reshape(noise_stft_mag_features, (129, 8, 1), name="noise_stft_mag_features")
    clean_stft_magnitude = tf.reshape(clean_stft_magnitude, (129, 1, 1), name="clean_stft_magnitude")
    noise_stft_phase = tf.reshape(noise_stft_phase, (129,), name="noise_stft_phase")

    return noise_stft_mag_features, clean_stft_magnitude

In [8]:
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, Flatten, Dense, Reshape, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras import Model, Sequential

In [37]:

train_dataset = tf.data.TFRecordDataset([train_tfrecords_filenames])
train_dataset = train_dataset.map(tf_record_parser)
#train_dataset = train_dataset.shuffle(5)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(512)
#train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

features:  {'clean_stft_magnitude': <tf.Tensor 'ParseSingleExample/ParseSingleExample:0' shape=() dtype=string>, 'noise_stft_mag_features': <tf.Tensor 'ParseSingleExample/ParseSingleExample:1' shape=() dtype=string>, 'noise_stft_phase': <tf.Tensor 'ParseSingleExample/ParseSingleExample:2' shape=() dtype=string>}


In [38]:

test_dataset = tf.data.TFRecordDataset([val_tfrecords_filenames])
test_dataset = test_dataset.map(tf_record_parser)
test_dataset = test_dataset.repeat(1)
test_dataset = test_dataset.batch(512)

features:  {'clean_stft_magnitude': <tf.Tensor 'ParseSingleExample/ParseSingleExample:0' shape=() dtype=string>, 'noise_stft_mag_features': <tf.Tensor 'ParseSingleExample/ParseSingleExample:1' shape=() dtype=string>, 'noise_stft_phase': <tf.Tensor 'ParseSingleExample/ParseSingleExample:2' shape=() dtype=string>}


In [11]:
import keras

Using TensorFlow backend.


In [12]:
def conv_block(x, filters, kernel_size, strides, padding='same', use_bn=True):
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(0.0006))(x)
    x = Activation('relu')(x)
    if use_bn:
        x = BatchNormalization()(x)
    return x

In [13]:
def full_pre_activation_block(x, filters, kernel_size, strides, padding='same', use_bn=True):
    shortcut = x
    in_channels = x.shape[-1]

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=in_channels, kernel_size=kernel_size, strides=strides, padding='same')(x)

    return shortcut + x

In [14]:
from tensorflow.python.keras.layers import Lambda;

def norm(fc2):

    fc2_norm = tf.keras.backend.l2_normalize(fc2, axis = 3)

    illum_est = tf.reduce_sum(fc2_norm, axis = (1, 2))

    illum_est = tf.keras.backend.l2_normalize(illum_est)

    return illum_est



In [15]:
tf.keras.backend.clear_session()
tf.compat.v1.disable_eager_execution()
def build_model(l2_strength):
    inputs = Input(shape=(numFeatures,numSegments,1))
    x = inputs
    print("Input Shape: ", x.shape)
  # -----
    x = tf.keras.layers.ZeroPadding2D(((4,4), (0,0)))(x)
    x = Conv2D(filters=18, kernel_size=[9,8], strides=[1, 1], padding='valid', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

    skip0 = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
                 kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(skip0)
    x = BatchNormalization()(x)

    x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

  # -----
    x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

    skip1 = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
                 kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(skip1)
    x = BatchNormalization()(x)

    x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    # ----
    x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

  # ----
    x = tf.keras.layers.SpatialDropout2D(0.2)(x)
    x = Conv2D(filters=1, kernel_size=[129,1], strides=[1, 1], padding='same')(x)
   # out = Lambda(norm)(x)
    print(x.shape);
    
    model = Model(inputs=inputs, outputs=x)

    optimizer = tf.keras.optimizers.Adam(3e-4)
  #optimizer = RAdam(total_steps=10000, warmup_proportion=0.1, min_lr=3e-4)

    model.compile(optimizer=optimizer, loss='mse')
    return model

In [16]:

model = build_model(l2_strength=0.0)
model.summary()

Input Shape:  (?, 129, 8, 1)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 129, 1, 1)
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 129, 8, 1)         0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 137, 8, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 129, 1, 18)        1296      
_________________________________________________________________
activation (Activation)      (None, 129, 1, 18)        0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 129, 1, 18)        72    

In [17]:

def l2_norm(vector):
    return np.square(vector)

def SDR(denoised, cleaned, eps=1e-7): # Signal to Distortion Ratio
    a = l2_norm(denoised)
    b = l2_norm(denoised - cleaned)
    a_b = a / b
    return np.mean(10 * np.log10(a_b + eps))

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq='batch')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='denoiser_cnn_log_mel_generator.h5', 
                                                         monitor='val_loss', save_best_only=True)
#print(len(tf.data.train_dataset))
train_dataset = train_dataset.apply(tf.data.experimental.ignore_errors())


#clear_tf_session()
print("hiiiiiiiiiiiiiiiiiiiiiii ",train_dataset)
model.fit(train_dataset,
          steps_per_epoch=2,
          validation_data=test_dataset,
          epochs=3,
          callbacks=[early_stopping_callback, tensorboard_callback]
         )

hiiiiiiiiiiiiiiiiiiiiiii  <DatasetV1Adapter shapes: ((?, 129, 8, 1), (?, 129, 1, 1)), types: (tf.float32, tf.float32)>
Epoch 1/3
